In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

ImportError: cannot import name 'saveable_objects_from_trackable' from 'tensorflow.python.training.saving.saveable_object_util' (C:\Users\CSAS\anaconda3\envs\gpu2\lib\site-packages\tensorflow\python\training\saving\saveable_object_util.py)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
path = './clothing-dataset-small/train/t-shirt'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
fullname = f'{path}/{name}'
load_img(fullname)

In [ ]:
img = load_img(fullname, target_size=(299, 299))

In [ ]:
x = np.array(img)
x.shape

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [ ]:
X = np.array([x])

In [ ]:
X = preprocess_input(X)

In [ ]:
pred = model.predict(X)

In [ ]:
pred.shape

In [ ]:
decode_predictions(pred)

In [ ]:
#Higher values in feature map equate to higher simlilarity to the image when filtered.
#integrate a multitude of feature maps for an increase in complexity   

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train', 
    target_size =(150, 150),
    batch_size =32)

In [ ]:
X, y = next(train_ds)

In [ ]:
train_ds.class_indices

In [ ]:
y[:5]

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train', 
    target_size =(150, 150),
    batch_size =32)
val_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

val_ds = val_gen.flow_from_directory( 
    './clothing-dataset-small/validation', 
    target_size =(150, 150),
    batch_size =32,
    shuffle = False
)


In [ ]:
#establishing a model

base_model = Xception(
    weights= 'imagenet', 
    include_top= False,
    input_shape= (150,150,3)
)
base_model.trainable =False

In [ ]:
#establishing a model
base_model = Xception(
    weights= 'imagenet', 
    include_top= False,
    input_shape= (150,150,3)
)
base_model.trainable =False
#model recieves images, replacing the top layer filter
inputs = keras.Input(shape=(150,150,3))

base = base_model(inputs)

vectors = keras.layers.GlobalAveragePooling2D()(base)

outputs = keras.layers.Dense(10)(vectors)

model = keras.Model(inputs, outputs)

In [ ]:
preds = model.predict(X)

In [ ]:
preds.shape

In [ ]:
# # from tensorflow import keras
# # from tensorflow.keras import layers

# # model = keras.Sequential()
# # model.add(layers.Dense(64, kernel_initializer='uniform', input_shape=(10,)))
# # model.add(layers.Activation('softmax'))

# # opt = keras.optimizers.Adam(learning_rate=0.01)
# # model.compile(loss='categorical_crossentropy', optimizer=opt)
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# # Instantiate an optimizer.
# optimizer = tf.keras.optimizers.Adam()

# # Iterate over the batches of a dataset.
# for x, y in dataset:
#     # Open a GradientTape.
#     with tf.GradientTape() as tape:
#         # Forward pass.
#         logits = model(x)
#         # Loss value for this batch.
#         loss_value = loss_fn(y, logits)

#     # Get gradients of loss wrt the weights.
#     gradients = tape.gradient(loss_value, model.trainable_weights)

#     # Update the weights of the model.
#     optimizer.apply_gradients(zip(gradients, model.trainable_weights))
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-2,
#     decay_steps=10000,
#     decay_rate=0.9)
# optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
plt.plot(history.history ['accuracy'], label='train')
plt.plot(history.history ['val_accuracy'], label = 'val')
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
#learning rate model function;

def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(
    weights= 'imagenet', 
    include_top= False,
    input_shape= (150,150,3)
    )
    base_model.trainable =False
    
########################################
    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    outputs = keras.layers.Dense(10)(vectors)
    model = keras.Model(inputs, outputs)
########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model

In [ ]:
scores = {}

for lr in [0.0001, 0.001, 0.01, 0.1]:
    print (lr)
    
    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    
    print()
    print()

In [ ]:
del scores[0.1]
del scores[0.0001]

In [ ]:
for lr, hist in scores.items():
#     plt.plot(hist['accuracy'], label='train=%s' % lr)
    plt.plot(hist['val_accuracy'], label='val=%s' % lr)

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001

In [ ]:
model.save_weights('model_v1.h5', save_format='h5')

In [ ]:
#invoking callback checkpoints from model evaluation 
checkpoint=keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5', 
    save_best_only=True,
    monitor= 'val_accuracy',
    mode='max'
                               )

In [ ]:
learning_rate = 0.001 
    
model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, 
        epochs=10, 
        validation_data=val_ds,
        callbacks=[checkpoint]
)

In [ ]:
learning_rate = 0.01 
    
model = make_model(learning_rate=learning_rate)
history = model.fit(train_ds, 
        epochs=10, 
        validation_data=val_ds,
        callbacks=[checkpoint]
)

In [ ]:
learning_rate= 0.001

scores = {}

for size in [10, 100, 1000]:
    print(size)
    
    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history
    
    print()
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % size))

plt.xticks(np.arange(10))
# plt.yticks[(0.78, 0.80, 0.82, 0.825, 0.83)]
plt.legend()

In [ ]:
#learning rate model function;

def make_model(learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
    weights= 'imagenet', 
    include_top= False,
    input_shape= (150,150,3)
    )
    base_model.trainable =False
    
########################################
    inputs = keras.Input(shape=(150,150,3))
    base = base_model(inputs)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
 #########################################
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
 ##########################################    
    outputs = keras.layers.Dense(10)(vectors)
    model = keras.Model(inputs, outputs)
########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model 

In [ ]:
learning_rate= 0.001
size=100

scores = {}

for droprate in [0.0, 0.2, 0.5, 0.8]:
    print(droprate)
    
    model = make_model(learning_rate=learning_rate, 
                       size_inner=size, 
                       droprate=droprate
                      )
    
    history = model.fit(train_ds, epochs=30, validation_data=val_ds)
    scores[droprate] = history.history
    
    print()
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % droprate))

plt.ylim(0.78, 0.86)
plt.legend()

In [ ]:
hist = scores[0.0]
plt.plot(hist['val_accuracy'], label=0.0)

hist = scores[0.2]
plt.plot(hist['val_accuracy'], label=0.2)
plt.legend()
#plt.plot(hist['accuracy'], label=(['val=%s' % droprate]))

In [ ]:
import numpy as np
import pandas as pd

from PIL import Image

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import tempfile
# model_dir = tempfile.mkdtemp()
# keras_estimator = tf.keras.estimator.model_to_estimator(
#     keras_model=model, model_dir=model_dir)

In [ ]:
def apply_transform(x, transform_parameters):
    x= tf.keras.preprocesssing.image.apply_affine_transform(x,
        transform_parameters.get('theta',0),
        transform_parameters.get('tx',0),
        transform_parameters.get('ty',0),
        transform_parameters.get('shear',0),
        transform_parameters.get('zx',0),
        transform_parameters.get('zy',0),
        row_axis=0,
        col_axis=1,
        channel_axis=2,
        fill_mode='nearest',
        cval=0.0,
        order=1
)
    if transform_parameters.get('flip_horizontal', False):
        x=x[::-1, :, :]
    if transform_parameters.get('flip_vertical', False):
        x=x[:, ::-1, :]
    return x

In [ ]:
def argument(img, 
            rotation=0,
             height_shift=0,
             width_shift=0,
             shear=0, 
             zoom_x=1,
             zoom_y=1,
             flip_horizontal=0,
             flip_vertical=0):
             
        x=np.array(img)
             
        transform_parameters ={
        'theta': rotation,
        'tx': height_shift,
        'shear': shear,
        'zx': zoom_x,
        'zy': zoom_y,
        'flip_horizontal': flip_horizontal,
        'flip_vertical': flip_vertical,
    }
    
        xaug = apply_transform(x, transform_parameters)
        imgaug = Image.fromarray(xaug)
        return imgaug
             

In [ ]:
def show_augmentations(params, values, figsize=(20,6)):
    imgs = []
    
    for v in values:
        im = argument(tshirt, **{param: v})
        imgs.append(im)
        
    plt.figure(figsize=figsize)
    
    n= len(values)
    
    for i in range(n):
        plt.sublplot(1, n, i + i)
        plt.imshow(np.array(imgs[i]))
        plt.title('%s=%s'  % (param, values[1]))
        plt.axis('off')
    
    plt.show() 

In [ ]:
#load the image
tshirt = Image.open('./clothing-dataset-small/train/t-shirt/5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg')
tshirt = tshirt.resize((150, 150), resample=Image.NEAREST)
tshirt

In [ ]:
#verify varient augmentations 

In [ ]:
plt.figure(figsize=(10, 3))

plt.subplot(1, 4, 1)
plt.imshow(np.array(tshirt))
plt.title('Original')
plt.axis("off")


plt.subplot(1, 4, 2)

img = argument(tshirt, flip_horizontal=1)
plt.imshow(np.array(img))
plt.title('Horizontal')
plt.axis("off")


plt.subplot(1, 4, 3)

img = argument(tshirt, flip_vertical=1)
plt.imshow(np.array(img))
plt.title('Vertical')
plt.axis("off")


plt.subplot(1, 4, 4)

img = argument(tshirt, flip_horizontal=1, flip_vertical=1)
plt.imshow(np.array(img))
plt.title('Both')
plt.axis("off")

plt.show() 


In [ ]:
param = 'rotation'
values = [-30, -15, 0, 15, 30]
show_augmentations(param, values, (10, 3))

In [ ]:
param = 'height_shift'
values = [-30, -15, 0, 15, 30]
show_augmentations(param, values, (10, 3))

In [ ]:
param = 'width_shift'
values = [-30, -15, 0, 15, 30]
show_augmentations(param, values, (10, 3))

In [ ]:
param = 'zoom_x'
values = [0.25, 0.5, 1, 2, 4]
show_augmentations(param, values, (10, 3))

In [ ]:
vis_gen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=5.0,
    height_shift_range=5.0,
    shear_range=10.0,
    zoom_range=0.1,
    horizontal_flip=True,
)

In [ ]:
x = np.array(tshirt)

In [ ]:
plt.figure(figsize=(10, 4))

for i in range(10):
    t = vis_gen.get_random_transform((150, 150))
    xaug = vis_gen.apply_transform(x, t)

    plt.subplot(2, 5, i + 1)
    plt.imshow(xaug.astype('uint8'))
    plt.axis("off")

In [ ]:
values = [0.5, 0.75, 1, 1.2, 1.5]

imgs = []

for v in values:
    im = augment(tshirt, zoom_y=v, zoom_x=v)
    imgs.append(im)

plt.figure(figsize=(10, 3))

n = len(values)

for i in range(n):
    plt.subplot(1, n, i + 1)
    plt.imshow(np.array(imgs[i]))
    plt.title('%s=%s' % ('zoom', values[i]))
    plt.axis("off")

plt.show()

In [ ]:
#299x299 model size 

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10, 
    zoon_range=0.1,
    vertical_flip=False,
    
    
)
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train', 
    target_size =(150, 150),
    batch_size =32)
val_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

val_ds = val_gen.flow_from_directory( 
    './clothing-dataset-small/validation', 
    target_size =(150, 150),
    batch_size =32,
    shuffle = False
)


In [ ]:
learning_rate= 0.001
size=100
droprate = 0.2
    
    model = make_model(learning_rate=learning_rate, 
                       size_inner=size, 
                       droprate=droprate
                      )
    
    history = model.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
    weights= 'imagenet', 
    include_top= False,
    input_shape= (input_size, input_size, 3)
    )
    base_model.trainable =False
    
########################################
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
 #########################################
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
 ##########################################    
    outputs = keras.layers.Dense(10)(vectors)
    model = keras.Model(inputs, outputs)
########################################
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
 
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model 

In [ ]:
input_size=299

In [ ]:
train_gen = ImageDataGenerator(prepr cessing_function=preprocess_input,
                              shear_range=10,
                              zoom_range=0.1,
                                 horizontal_flip=True
                              )
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train', 
    target_size= (input_size, input_size),
    batch_size =32
)

val_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

val_ds = val_gen.flow_from_directory( 
    './clothing-dataset-small/validation', 
    target_size =(input_size, input_size),
    batch_size =32,
    shuffle = False
)


In [ ]:
checkpoint=keras.callbacks.ModelCheckpoint(
    'xception_v4_{epoch:02d}_{val_accuracy:.3f}.h5', 
    save_best_only=True,
    monitor= 'val_accuracy',
    mode='max'
                               )

In [ ]:
learning_rate= 0.0005
size=100
droprate = 0.2
    

    model = make_model(input_size=input_size,
                       learning_rate=learning_rate, 
                       size_inner=size, 
                       droprate=droprate
                      )
    
    history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[checkpoint])

## Implementing the Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = keras.models.load_model('xception_v1_07_0.842.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input
                             
test_ds = test_gen.flow_from_directory(
    './clothing-dataset-small-validation',
    target_size=(299, 299),
    batch_size=32,
    shuffle=False
)